# Keras

This Keras tutorial takes you through the process of building a feedforward neural network using Keras Sequential API (although the same model can be built using Keras functional API, which provides a lower level control).

Most of the documentation here is shamelessly taken from [Keras Official Website](https://keras.io) which you are highly encouraged to explore.

- [Keras Sequential Model Guide](https://keras.io/getting-started/sequential-model-guide/)
- [Keras Sequential API](https://keras.io/models/sequential/)


### What is Keras?

Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Use Keras if you need a deep learning library that:

- Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).
- Supports both convolutional networks and recurrent networks, as well as combinations of the two.
- Runs seamlessly on CPU and GPU.

### Keras installation

Super easy when you have `pip`. Just work some `pip install` magic.

    pip install keras

<br/>

_Let's dive in._

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from keras import regularizers

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(1)

### Preparing Data

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [ ]:
x_train = train.drop("ProbabilityOfDefaultIn2Yrs", axis=1)
y_train = train["ProbabilityOfDefaultIn2Yrs"]
x_test = test.drop("ProbabilityOfDefaultIn2Yrs", axis=1)
y_test = test["ProbabilityOfDefaultIn2Yrs"]

In [ ]:
x_train.drop("ID", axis=1, inplace=True)
x_test.drop("ID", axis=1, inplace=True)

In [ ]:
x_train_final = x_train.values
x_test_final = x_test.values
y_train_final = y_train.values.reshape(y_train.count(), 1)
y_test_final = y_test.values.reshape(y_test.count(), 1)

In [ ]:
x_train_final.shape, y_train_final.shape

In [ ]:
x_test_final.shape, y_test_final.shape

## Part I. A Gentle Intorduction to Dense Networks in Keras

### 1. Creating a Model

- The core data structure of Keras is a `model`, a way to organize layers. The simplest type of model is the `Sequential` model, a linear stack of layers.

        from keras.models import Sequential
<br/>

- You can create a `Sequential` model by passing a list of layer instances to the constructor

        model = Sequential([
            Dense(32, input_shape=(784,)),
            Activation('relu'),
            Dense(10),
            Activation('softmax'),
        ])
<br/>

- Or you can add layers via the .add() method

        model = Sequential()
        model.add(Dense(32, input_dim=784))
        model.add(Activation('relu'))
        model.add(Dense(10))
        model.add(Activation('softmax'))
<br/>

- Or if you're like me and prefer brevity

        model = Sequential()
        model.add(Dense(units=32, activation='relu', input_dim=784))
        model.add(Dense(units=10, activation='softmax'))

In [ ]:
model = Sequential()
model.add(Dense(units=5, activation='relu', input_dim=10))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

### 2. Compiling a Model

Before training a model, you need to configure the learning process, which is done via the `compile` method. It receives three arguments:

- __An optimizer:__ This could be the string identifier of an existing optimizer (such as `rmsprop` or `adagrad`), or an instance of the `Optimizer` class. [More on Optimizers here.](https://keras.io/optimizers).
<br/>

- __A loss function:__ This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as `categorical_crossentropy` or `mse`), or it can be an objective function. [More on losses here.](https://keras.io/losses)
<br/>

- __A list of metrics:__ For any classification problem you will want to set this to `metrics=['accuracy']`. A metric could be the string identifier of an existing metric or a custom metric function. [More on metrics here.](https://keras.io/metrics/)

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### 3. Training

Keras models are trained on Numpy arrays of input data and labels. For training a model, you will typically use the `fit` function and pass hyperparameters `[epochs, batch_size]` etc. as arguments.

In [ ]:
history = model.fit(x_train_final, y_train_final, epochs=10)

In [ ]:
history.history

In [ ]:
plt.plot(np.squeeze(history.history["loss"]))
plt.ylabel('cost')
plt.xlabel('iterations (per tens)')
plt.show()

___Specifying a learning rate___

In [ ]:
model = Sequential()
model.add(Dense(units=5, activation='relu', input_dim=10))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
learning_rate = 0.0001
opt = SGD(lr=learning_rate)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train_final, y_train_final, epochs=10)

In [ ]:
plt.plot(np.squeeze(history.history["loss"]))
plt.ylabel('cost')
plt.xlabel('iterations (per tens)')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

### 4. Predictions & Evaluation

- Keras provides separate functions to get logits and classes. To get logtis, use `predict` function. For classes, use `predict_classes` functions.

- To evaluate accuracy directly, use `evaluate` function which returns loss as well as accuracy.

In [ ]:
result_logits = model.predict(x_test_final)
result_logits

In [ ]:
result_classes = model.predict_classes(x_test_final)
result_classes

In [ ]:
results = model.evaluate(x_test_final, y_test_final)
print("\nThe accuracy on test set is {}".format(results[1]))

### 5. Saving & Loading a Model

You can use `model.save(filepath)` to save a Keras model into a single HDF5 file which will contain:

- the architecture of the model, allowing to re-create the model
- the weights of the model
- the training configuration (loss, optimizer)
- the state of the optimizer, allowing to resume training exactly where you left off.
- You can then use `keras.models.load_model(filepath)` to reinstantiate your model. `load_model` will also take care of compiling the model using the saved training configuration (unless the model was never compiled in the first place).

[More on saving a model here.](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)

In [ ]:
model.save("saved_models/credit_score_basic.h5")

In [ ]:
del model

In [ ]:
model.summary()

In [ ]:
model = load_model("saved_models/credit_score_basic.h5")

In [ ]:
model.summary()